In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'n2o': 3.2e-07}
band = [9]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.017600                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.002038
0.000750      2.0 -0.015991                  2.0  0.002835
0.001052      3.0 -0.012460                  3.0  0.005036
0.001476      4.0 -0.008510                  4.0  0.007317
0.002070      5.0 -0.004150                  5.0  0.009832
0.002904      6.0  0.000658                  6.0  0.012611
0.004074      7.0  0.005921                  7.0  0.015676
0.005714      8.0  0.011687                  8.0  0.019056
0.008015      9.0  0.017991                  9.0  0.022772
0.011243     10.0  0.024868                 10.0  0.026860
0.015771     11.0  0.032348                 11.0  0.031343
0.022122     12.0  0.040467                 12.0  0.036256
0.031031     13.0  0.049211                 13.0  0.041615
0.043528     14.0  0.057412                 14.0  0.046765
0.061057     15.0  0.063819                 15.0  0.051021
0.085645     16.0  0.069295                 16.0  0.054900
0.120136     17.0  0.074747                 17.0  0.058975
0.168516     18.0  0.080183                 18.0  0.063263
0.236378     19.0  0.085623                 19.0  0.067766
0.331549     20.0  0.091170                 20.0  0.072476
0.465100     21.0  0.096298                 21.0  0.076930
0.652400     22.0  0.088438                 22.0  0.073397
0.915100     23.0  0.065686                 23.0  0.060106
1.283650     24.0  0.043782                 24.0  0.046009
1.800600     25.0  0.027670                 25.0  0.034134
2.525700     26.0  0.016391                 26.0  0.024181
3.542800     27.0  0.008839                 27.0  0.015946
4.969550     28.0  0.004860                 28.0  0.009978
6.970850     29.0  0.003604                 29.0  0.006261
9.778100     30.0  0.003073                 30.0  0.003690
13.715850    31.0  0.002356                 31.0  0.001636
19.239350    32.0  0.001424                 32.0  0.000076
26.987250    33.0  0.001088                 33.0 -0.000514
37.855300    34.0  0.001454                 34.0 -0.000309
53.100050    35.0  0.001685                 35.0 -0.000185
73.887500    36.0  0.001706                 36.0 -0.000383
97.662500    37.0  0.001493                 37.0 -0.000925
121.437500   38.0  0.000783                 38.0 -0.001875
145.212500   39.0 -0.000105                 39.0 -0.002887
168.987500   40.0 -0.001051                 40.0 -0.003935
192.762500   41.0 -0.002111                 41.0 -0.005028
216.537500   42.0 -0.003373                 42.0 -0.006212
240.312500   43.0 -0.004850                 43.0 -0.007441
264.087500   44.0 -0.006625                 44.0 -0.008743
287.862500   45.0 -0.008649                 45.0 -0.010041
311.637500   46.0 -0.009491                 46.0 -0.010296
335.412500   47.0 -0.008077                 47.0 -0.008719
359.187500   48.0 -0.005453                 48.0 -0.006073
382.962500   49.0 -0.002020                 49.0 -0.002637
406.737500   50.0  0.002414                 50.0  0.001805
430.512500   51.0  0.007475                 51.0  0.006960
454.287500   52.0  0.013045                 52.0  0.012391
478.062500   53.0  0.019025                 53.0  0.017974
501.837500   54.0  0.025343                 54.0  0.024086
525.612500   55.0  0.031904                 55.0  0.030904
549.387500   56.0  0.038557                 56.0  0.037895
573.162500   57.0  0.045140                 57.0  0.044397
596.937500   58.0  0.051426                 58.0  0.050164
620.712500   59.0  0.057201                 59.0  0.055326
644.487500   60.0  0.062277                 60.0  0.060219
668.262500   61.0  0.066539                 61.0  0.064774
692.037500   62.0  0.069980                 62.0  0.069004
715.812500   63.0  0.072688                 63.0  0.072584
739.587500   64.0  0.074874                 64.0  0.075494
76

# Fluxes by Layer

CRD                        CLIRAD                \
                        flug      fldg     fnetg      flug          fldg   
pressure     level                                                         
0.000000e+00 1     -7.841210  0.000000 -7.841210       NaN           NaN   
1.000000e-08 1           NaN       NaN       NaN -7.887977  9.417991e-08   
6.244000e-04 2     -7.841213  0.000001 -7.841212 -7.887977  7.251869e-07   
8.759000e-04 3     -7.841213  0.000002 -7.841212 -7.887977  9.987250e-07   
1.228600e-03 4     -7.841215  0.000002 -7.841213 -7.887978  1.430097e-06   
1.723400e-03 5     -7.841216  0.000003 -7.841213 -7.887978  2.100681e-06   
2.417400e-03 6     -7.841219  0.000005 -7.841213 -7.887978  3.143631e-06   
3.390900e-03 7     -7.841221  0.000008 -7.841213 -7.887978  4.766644e-06   
4.756500e-03 8     -7.841225  0.000012 -7.841212 -7.887978  7.293153e-06   
6.672000e-03 9     -7.841229  0.000019 -7.841210 -7.887978  1.122517e-05   
9.358900e-03 10    -7.841234  0.000030 -7.841204 -7.887977  1.734029e-05   
1.312780e-02 11    -7.841239  0.000046 -7.841193 -7.887974  2.684356e-05   
1.841450e-02 12    -7.841244  0.000071 -7.841173 -7.887969  4.159740e-05   
2.583020e-02 13    -7.841247  0.000110 -7.841137 -7.887960  6.447889e-05   
3.623230e-02 14    -7.841247  0.000170 -7.841076 -7.887945  9.991299e-05   
5.082340e-02 15    -7.841238  0.000261 -7.840977 -7.887918  1.540994e-04   
7.129060e-02 16    -7.841218  0.000396 -7.840822 -7.887875  2.352695e-04   
1.000000e-01 17    -7.841178  0.000592 -7.840587 -7.887809  3.556720e-04   
1.402710e-01 18    -7.841106  0.000876 -7.840230 -7.887706  5.341104e-04   
1.967600e-01 19    -7.840979  0.001286 -7.839694 -7.887547  7.982911e-04   
2.759970e-01 20    -7.840761  0.001871 -7.838890 -7.887301  1.188856e-03   
3.871000e-01 21    -7.840391  0.002701 -7.837690 -7.886923  1.764853e-03   
5.431000e-01 22    -7.839768  0.003857 -7.835911 -7.886346  2.608963e-03   
7.617000e-01 23    -7.838893  0.005273 -7.833621 -7.885569  3.733131e-03   
1.068500e+00 24    -7.837932  0.006699 -7.831233 -7.884696  5.044785e-03   
1.498800e+00 25    -7.836980  0.007978 -7.829002 -7.883793  6.487027e-03   
2.102400e+00 26    -7.836069  0.009046 -7.827023 -7.882896  8.031187e-03   
2.949000e+00 27    -7.835251  0.009872 -7.825379 -7.882064  9.624107e-03   
4.136600e+00 28    -7.834618  0.010482 -7.824136 -7.881385  1.118871e-02   
5.802500e+00 29    -7.834229  0.011052 -7.823177 -7.880926  1.269879e-02   
8.139200e+00 30    -7.834065  0.011885 -7.822179 -7.880719  1.422529e-02   
1.141700e+01 31    -7.834199  0.013213 -7.820986 -7.880903  1.584187e-02   
1.601470e+01 32    -7.834907  0.015205 -7.819703 -7.881828  1.765809e-02   
2.246400e+01 33    -7.836742  0.018127 -7.818615 -7.884086  1.997375e-02   
3.151050e+01 34    -7.840222  0.022774 -7.817449 -7.888323  2.365969e-02   
4.420010e+01 35    -7.845722  0.030458 -7.815263 -7.895288  3.016009e-02   
6.200000e+01 36    -7.854203  0.042493 -7.811710 -7.906577  4.105815e-02   
8.577500e+01 37    -7.866709  0.059805 -7.806904 -7.924061  5.746356e-02   
1.095500e+02 38    -7.880566  0.077866 -7.802700 -7.944152  7.494846e-02   
1.333250e+02 39    -7.896347  0.095852 -7.800496 -7.967166  9.268238e-02   
1.571000e+02 40    -7.914934  0.114144 -7.800791 -7.993636  1.110194e-01   
1.808750e+02 41    -7.936631  0.132880 -7.803752 -8.023587  1.298872e-01   
2.046500e+02 42    -7.961843  0.152147 -7.809696 -8.057129  1.492682e-01   
2.284250e+02 43    -7.990891  0.171696 -7.819196 -8.094265  1.689082e-01   
2.522000e+02 44    -8.024340  0.191484 -7.832856 -8.135115  1.887999e-01   
2.759750e+02 45    -8.063107  0.211593 -7.851514 -8.180007  2.090651e-01   
2.997500e+02 46    -8.108282  0.232411 -7.875871 -8.229283  2.300615e-01   
3.235250e+02 47    -8.159085  0.256483 -7.902602 -8.281992  2.537695e-01   
3.473000e+02 48    -8.212869  0.287519 -7.925349 -8.336043  2.832615e-01   
3.710750e+02 49    -8.267851  0.327143 -7.940708 -8.390044  3.201570e-01 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')